<a href="https://colab.research.google.com/github/liuyixi520/code/blob/main/ernie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 【快速上手ERNIE 3.0】中文情感分析、法律文本多标签分类、中文语义匹配、MSRA序列标注项目实战



## ERNIE 3.0模型介绍
ERNIE 3.0首次在百亿级预训练模型中引入大规模知识图谱，提出了海量无监督文本与大规模知识图谱的平行预训练方法(Universal Knowledge-Text Prediction)，通过将知识图谱挖掘算法得到五千万知识图谱三元组与4TB大规模语料同时输入到预训练模型中进行联合掩码训练，促进了结构化知识和无结构文本之间的信息共享，大幅提升了模型对于知识的记忆和推理能力。

ERNIE 3.0框架分为两层。第一层是通用语义表示网络，该网络学习数据中的基础和通用的知识。第二层是任务语义表示网络，该网络基于通用语义表示，学习任务相关的知识。在学习过程中，任务语义表示网络只学习对应类别的预训练任务，而通用语义表示网络会学习所有的预训练任务。

![](https://ai-studio-static-online.cdn.bcebos.com/b00b30257fcf44ad98f37e496702b6dc7a0d32ffb1de469b94f56829ed7766aa)
<font size=2><center>ERNIE 3.0模型框架</center></font>

[comment]: <> (ERNIE 3.0介绍参考新闻稿 http://ex.chinadaily.com.cn/exchange/partners/82/rss/channel/cn/columns/snl9a7/stories/WS60e41d0fa3101e7ce9758648.html)


In [ ]:
#基础环境准备
!pip install --upgrade paddlenlp

import os
import paddle
import paddlenlp

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 1.4MB 6.0MB/s eta 0:00:01
     |████████████████████████████████| 368kB 9.1MB/s eta 0:00:01
     |████████████████████████████████| 2.7MB 8.9MB/s eta 0:00:01
     |████████████████████████████████| 92kB 53.5MB/s eta 0:00:01
     |████████████████████████████████| 102kB 474kB/s eta 0:00:01
     |████████████████████████████████| 112kB 408kB/s eta 0:00:01
     |████████████████████████████████| 28.0MB 5.5MB/s eta 0:00:01
     |████████████████████████████████| 1.1MB 10.9MB/s eta 0:00:01
     |████████████████████████████████| 81kB 3.5MB/s eta 0:00:011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 143kB 55.5MB/s eta 0:00:01
     |████████████████████████████████| 92kB 2.9MB/s eta 0:00:011
     |████████████████████████████████| 51kB 2.1MB/s eta 0:00:011
  Installing build

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from numpy import (exp, inf, pi, sqrt, floor, sin, cos, around, int,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/scipy/linalg/__init__.py:217: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func


# 一. 中文情感分析实战
本项目将介绍如何基于PaddleNLP利用ERNIE 3.0预训练模型微调并进行中文情感分析预测，主要包括“什么是情感分析任务”、“ERNIE 3.0模型”、“如何使用ERNIE 3.0中文预训练模型进行句子级别情感分析”等三个部分。


## 1. 什么是情感分析任务

人类的自然语言蕴含着丰富的情感色彩，语言可以表达情绪（如悲伤、快乐）、心情（如倦怠、忧郁）、喜好（如喜欢、讨厌）、个性特征和立场等等。在互联网大数据时代，人类比以往任何时候都更公开地表达自己的想法和感受，如何快速地监控和理解所有类型数据中情绪变得尤为重要。情感分析是一种自然语言处理 (NLP) 技术，用于确定数据情感是正面的、负面的还是中性的。情感分析通常在文本数据上进行，在商品喜好、消费决策、舆情分析等场景中均有应用。利用机器自动分析这些情感倾向，不但有助于帮助企业监控客户反馈中的品牌和产品情感，并了解客户需求，还有助于企业分析商业伙伴们的态度，以便更好地进行商业决策。

生活中常见将一句话或一段文字的进行情感标记，如标记为正向、负向、中性的三分类问题，这属于句子级别情感分析任务。此外常见的情感分析任务还包括词级别情感分析和目标级别情感分析。

![](https://ai-studio-static-online.cdn.bcebos.com/49480ab6429b45049faf22febd0abace2cabda1630574cfb990c4fef4fecd453)
<font size=2><center>句子的情感被标记为正向、负向或中性</center></font>





## 2. 如何使用ERNIE 3.0中文预训练模型进行句子级别情感分析



### 2.1 加载中文情感分析数据集ChnSentiCorp

ChnSentiCorp是中文句子级情感分类数据集，包含酒店、笔记本电脑和书籍的网购评论，数据集示例：
```
 qid	label	text_a
 0	1	這間酒店環境和服務態度亦算不錯,但房間空間太小~~不宣容納太大件行李~~且房間格調還可以~~ 中餐廳的廣東點心不太好吃~~要改善之~~~~但算價錢平宜~~可接受~~ 西餐廳格調都很好~~但吃的味道一般且令人等得太耐了~~要改善之~~
 1	<荐书> 推荐所有喜欢<红楼>的红迷们一定要收藏这本书,要知道当年我听说这本书的时候花很长时间去图书馆找和借都没能如愿,所以这次一看到当当有,马上买了,红迷们也要记得备货哦!
 2	0	商品的不足暂时还没发现，京东的订单处理速度实在.......周二就打包完成，周五才发货...
 ...
```
其中1表示正向情感，0表示负向情感，PaddleNLP已经内置该数据集，一键即可加载。更多数据集自定方法详见[如何自定义数据集](https://paddlenlp.readthedocs.io/zh/latest/data_prepare/dataset_self_defined.html)。

In [ ]:
#加载中文评论情感分析语料数据集ChnSentiCorp
from paddlenlp.datasets import load_dataset

train_ds, dev_ds, test_ds = load_dataset("chnsenticorp", splits=["train", "dev", "test"])

# 数据集返回为MapDataset类型
print("数据类型:", type(train_ds))
# label代表标签，qid代表数据编号，测试集中不包含标签信息
print("训练集样例:", train_ds[0])
print("验证集样例:", dev_ds[0])
print("测试集样例:", test_ds[0])

100%|██████████| 1909/1909 [00:00<00:00, 5568.13it/s]


数据类型: <class 'paddlenlp.datasets.dataset.MapDataset'>
训练集样例: {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1, 'qid': ''}
验证集样例: {'text': '這間酒店環境和服務態度亦算不錯,但房間空間太小~~不宣容納太大件行李~~且房間格調還可以~~ 中餐廳的廣東點心不太好吃~~要改善之~~~~但算價錢平宜~~可接受~~ 西餐廳格調都很好~~但吃的味道一般且令人等得太耐了~~要改善之~~', 'label': 1, 'qid': '0'}
测试集样例: {'text': '这个宾馆比较陈旧了，特价的房间也很一般。总体来说一般', 'label': '', 'qid': '0'}


### 2.2 加载中文ERNIE 3.0预训练模型和分词器

PaddleNLP中Auto模块（包括AutoModel, AutoTokenizer及各种下游任务类）提供了方便易用的接口，无需指定模型类别，即可调用不同网络结构的预训练模型。PaddleNLP的预训练模型可以很容易地通过from_pretrained()方法加载，[Transformer预训练模型汇总](https://paddlenlp.readthedocs.io/zh/latest/model_zoo/index.html#transformer)包含了40多个主流预训练模型，500多个模型权重。


AutoModelForSequenceClassification可用于句子级情感分析和目标级情感分析任务，通过预训练模型获取输入文本的表示，之后将文本表示进行分类。PaddleNLP已经实现了ERNIE 3.0预训练模型，可以通过一行代码实现ERNIE 3.0预训练模型和分词器的加载。

In [ ]:
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "ernie-3.0-base-zh"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_classes=len(train_ds.label_list))
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2022-06-17 21:12:43,382] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-3.0-base-zh'.
[2022-06-17 21:12:43,385] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_base_zh.pdparams and saved to /home/work/.paddlenlp/models/ernie-3.0-base-zh
[2022-06-17 21:12:43,388] [    INFO] - Downloading ernie_3.0_base_zh.pdparams from https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_base_zh.pdparams
100%|██████████| 452M/452M [00:12<00:00, 37.3MB/s] 
[2022-06-17 21:13:09,519] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'ernie-3.0-base-zh'.
[2022-06-17 21:13:09,522] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_base_zh_vocab.txt and saved to /home/work/.paddlenlp/models/ernie-3.0-base-zh
[2022-06-17 21:13:09,524] [    INFO] - Downloading ernie_3.0_base_zh_v


### 2.3 基于预训练模型的数据处理
`Dataset`中通常为原始数据，需要经过一定的数据处理并进行采样组batch。
* 通过`Dataset`的`map`函数，使用分词器将数据集从原始文本处理成模型的输入。
* 定义`paddle.io.BatchSampler`和`collate_fn`构建 `paddle.io.DataLoader`。

实际训练中，根据显存大小调整批大小`batch_size`和文本最大长度`max_seq_length`。

In [ ]:
import functools
import numpy as np

from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding

# 数据预处理函数，利用分词器将文本转化为整数序列
def preprocess_function(examples, tokenizer, max_seq_length, is_test=False):

    result = tokenizer(text=examples["text"], max_seq_len=max_seq_length)
    if not is_test:
        result["labels"] = examples["label"]
    return result

trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128)
train_ds = train_ds.map(trans_func)
dev_ds = dev_ds.map(trans_func)

# collate_fn函数构造，将不同长度序列充到批中数据的最大长度，再将数据堆叠
collate_fn = DataCollatorWithPadding(tokenizer)

# 定义BatchSampler，选择批大小和是否随机乱序，进行DataLoader
train_batch_sampler = BatchSampler(train_ds, batch_size=32, shuffle=True)
dev_batch_sampler = BatchSampler(dev_ds, batch_size=64, shuffle=False)
train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
dev_data_loader = DataLoader(dataset=dev_ds, batch_sampler=dev_batch_sampler, collate_fn=collate_fn)


### 2.4 数据训练和评估
定义训练所需的优化器、损失函数、评价指标等，就可以开始进行预模型微调任务。

In [ ]:
# Adam优化器、交叉熵损失函数、accuracy评价指标
optimizer = paddle.optimizer.AdamW(learning_rate=2e-5, parameters=model.parameters())
criterion = paddle.nn.loss.CrossEntropyLoss()
metric = paddle.metric.Accuracy()

In [ ]:
# 开始训练
import time
import paddle.nn.functional as F

from demo.emotionAnalysisCn.eval import evaluate

epochs = 5 # 训练轮次
ckpt_dir = "/home/work/PretrainedModel/emotionAnalysisCn" #训练过程中保存模型参数的文件夹
best_acc = 0
best_step = 0
global_step = 0 #迭代次数
tic_train = time.time()
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # 计算模型输出、损失函数值、分类概率值、准确率
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.softmax(logits, axis=1)
        correct = metric.compute(probs, labels)
        metric.update(correct)
        acc = metric.accumulate()

        # 每迭代10次，打印损失函数值、准确率、计算速度
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, acc,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # 反向梯度回传，更新参数
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()

        # 每迭代100次，评估当前训练的模型、保存当前模型参数和分词器的词表等
        if global_step % 100 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            print(global_step, end=' ')
            acc_eval = evaluate(model, criterion, metric, dev_data_loader)
            if acc_eval > best_acc:
                best_acc = acc_eval
                best_step = global_step

                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)

模型训练过程中会输出如下日志:
```
global step 10, epoch: 1, batch: 10, loss: 0.66181, accu: 0.55000, speed: 4.53 step/s
global step 20, epoch: 1, batch: 20, loss: 0.54043, accu: 0.60938, speed: 4.92 step/s
global step 30, epoch: 1, batch: 30, loss: 0.42240, accu: 0.67708, speed: 4.88 step/s
global step 40, epoch: 1, batch: 40, loss: 0.34822, accu: 0.72266, speed: 4.86 step/s
global step 50, epoch: 1, batch: 50, loss: 0.31792, accu: 0.74438, speed: 4.85 step/s
global step 60, epoch: 1, batch: 60, loss: 0.36544, accu: 0.76719, speed: 4.86 step/s
global step 70, epoch: 1, batch: 70, loss: 0.19064, accu: 0.78795, speed: 4.87 step/s
global step 80, epoch: 1, batch: 80, loss: 0.32033, accu: 0.79883, speed: 4.86 step/s
global step 90, epoch: 1, batch: 90, loss: 0.22526, accu: 0.81007, speed: 4.82 step/s
global step 100, epoch: 1, batch: 100, loss: 0.30424, accu: 0.81781, speed: 4.85 step/s
100 eval loss: 0.25176, accuracy: 0.91167
[2022-05-13 17:07:09,935] [    INFO] - tokenizer config file saved in ernie_ckpt_1/tokenizer_config.json
[2022-05-13 17:07:09,938] [    INFO] - Special tokens file saved in ernie_ckpt_1/special_tokens_map.json
...
```
训练5个epoch预计需要7分钟。

In [ ]:
from demo.emotionAnalysisCn.eval import evaluate
# 加载ERNIR 3.0最佳模型参数
params_path = 'PretrainedModel/emotionAnalysisCn/model_state.pdparams'
state_dict = paddle.load(params_path)
model.set_dict(state_dict)
print('ERNIE 3.0 在ChnSentiCorp的dev集表现', end=' ')
eval_acc = evaluate(model, criterion, metric, dev_data_loader)

### 2.5 情感分析结果预测与保存
加载微调好的模型参数进行情感分析预测，并保存预测结果

In [ ]:
# 测试集数据预处理，利用分词器将文本转化为整数序列
trans_func_test = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128, is_test=True)
test_ds_trans = test_ds.map(trans_func_test)

# 进行采样组batch
collate_fn_test = DataCollatorWithPadding(tokenizer)
test_batch_sampler = BatchSampler(test_ds_trans, batch_size=32, shuffle=False)
test_data_loader = DataLoader(dataset=test_ds_trans, batch_sampler=test_batch_sampler, collate_fn=collate_fn_test)

In [ ]:
# 模型预测分类结果
import paddle.nn.functional as F

label_map = {0: '负面', 1: '正面'}
results = []
model.eval()
for batch in test_data_loader:
    input_ids, token_type_ids = batch['input_ids'], batch['token_type_ids']
    logits = model(batch['input_ids'], batch['token_type_ids'])
    probs = F.softmax(logits, axis=-1)
    idx = paddle.argmax(probs, axis=1).numpy()
    idx = idx.tolist()
    preds = [label_map[i] for i in idx]
    results.extend(preds)

In [ ]:
# 存储ChnSentiCorp预测结果  
test_ds = load_dataset("chnsenticorp", splits=["test"]) 

res_dir = "./results/emotionAnalysisCn"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
with open(os.path.join(res_dir, "ChnSentiCorp.tsv"), 'w', encoding="utf8") as f:
    f.write("qid\ttext\tprediction\n")
    for i, pred in enumerate(results):
        f.write(test_ds[i]['qid']+"\t"+test_ds[i]['text']+"\t"+pred+"\n")

ChnSentiCorp预测结果示例：

![](https://ai-studio-static-online.cdn.bcebos.com/581d255179974f6fa20f588fd4dafbbc0c8c7b1b27d54de19e0d77048ed2215a)


# 二.法律文本多标签分类实战
本项目将介绍如何基于PaddleNLP对ERNIE 3.0预训练模型微调完成法律文本多标签分类预测。本项目主要包括“什么是多标签文本分类预测”、“ERNIE 3.0模型”、“如何使用ERNIE 3.0中文预训练模型进行法律文本多标签分类预测”等三个部分。

## 1. 什么是多标签文本分类预测  


文本多标签分类是自然语言处理（NLP）中常见的文本分类任务，文本多标签分类在各种现实场景中具有广泛的适用性，例如商品分类、网页标签、新闻标注、蛋白质功能分类、电影分类、语义场景分类等。多标签数据集中样本用来自 `n_classes` 个可能类别的`m`个标签类别标记，其中`m`的取值在0到`n_classes`之间，这些类别具有不相互排斥的属性。通常，我们将每个样本的标签用One-hot的形式表示，正类用1表示，负类用0表示。例如，数据集中样本可能标签是A、B和C的多标签分类问题，标签为\[1,0,1\]代表存在标签 A 和 C 而标签 B 不存在的样本。

近年来，随着司法改革的全面推进，“以公开为原则，不公开为例外”的政策逐步确立，大量包含了案件事实及其适用法律条文信息的裁判文书逐渐在互联网上公开，海量的数据使自然语言处理技术的应用成为可能。法律条文的组织呈树形层次结构，现实中的案情错综复杂，同一案件可能适用多项法律条文，涉及数罪并罚，需要多标签模型充分学习标签之间的关联性，对文本进行分类预测。

## 2. 如何使用ERNIE 3.0中文预训练模型进行法律文本多标签分类预测



### 2.1 环境准备

In [ ]:
# 安装pyzmq
!pip install pyzmq==18.1.1


### 2.2 加载法律文本多标签数据

本数据集（[2019年法研杯要素识别任务](https://github.com/china-ai-law-challenge/CAIL2019/tree/master/%E8%A6%81%E7%B4%A0%E8%AF%86%E5%88%AB)）来自于“中国裁判文书网”公开的法律文书，每条训练数据由一份法律文书的案情描述片段构成，其中每个句子都被标记了对应的类别标签，数据集一共包含20个标签，标签代表含义如下：

```
DV1    0    婚后有子女
DV2    1    限制行为能力子女抚养
DV3    2    有夫妻共同财产
DV4    3    支付抚养费
DV5    4    不动产分割
DV6    5    婚后分居
DV7    6    二次起诉离婚
DV8    7    按月给付抚养费
DV9    8    准予离婚
DV10    9    有夫妻共同债务
DV11    10    婚前个人财产
DV12    11    法定离婚
DV13    12    不履行家庭义务
DV14    13    存在非婚生子
DV15    14    适当帮助
DV16    15    不履行离婚协议
DV17    16    损害赔偿
DV18    17    感情不和分居满二年
DV19    18    子女随非抚养权人生活
DV20    19    婚后个人财产
```
数据集示例：
```
text    labels
所以起诉至法院请求变更两个孩子均由原告抚养，被告承担一个孩子抚养费每月600元。	0,7,3,1
2014年8月原、被告因感情不和分居，2014年10月16日被告文某某向务川自治县人民法院提起离婚诉讼，被法院依法驳回了离婚诉讼请求。	6,5
女儿由原告抚养，被告每月支付小孩抚养费500元；	0,7,3,1
```
使用本地文件创建数据集，自定义`read_custom_data()`函数读取数据文件，传入`load_dataset()`创建数据集，返回数据类型为MapDataset。更多数据集自定方法详见[如何自定义数据集](https://paddlenlp.readthedocs.io/zh/latest/data_prepare/dataset_self_defined.html)。

In [ ]:
# 下载示例数据集，也可以导入数据集的方式导入数据集
%cd /home/work/data
!wget https://easydl-download.bj.bcebos.com/29129392321/demo_data.zip
!unzip -q demo_data.zip
!rm ./demo_data.zip

In [ ]:
# 数据集处理
import re

from paddlenlp.datasets import load_dataset

def clean_text(text):
    text = text.replace("\r", "").replace("\n", "")
    text = re.sub(r"\\n\n", ".", text)
    return text

# 定义读取数据集函数
def read_custom_data(is_test=False, is_one_hot=True):

    file_num = 6 if is_test else 48
    # 定义数据集和测试集路径，如果需要导入自定义数据集，请修改filepath即可
    filepath = '/home/work/data/raw_data/test/' if is_test else '/home/work/data/raw_data/train/'

    for i in range(file_num):
        f = open('{}labeled_{}.txt'.format(filepath, i))
        while True:
            line = f.readline()
            if not line:
                break
            data = line.strip().split('\t')
            # 标签用One-hot表示
            if is_one_hot:
                labels = [float(1) if str(i) in data[1].split(',') else float(0) for i in range(20)]
            else:
                labels = [int(d) for d in data[1].split(',')]
            yield {"text": clean_text(data[0]), "labels": labels}
        f.close()

#定义label意义映射,自定义数据集这里需要修改
label_vocab = {
    0: "婚后有子女",
    1: "限制行为能力子女抚养",
    2: "有夫妻共同财产",
    3: "支付抚养费",
    4: "不动产分割",
    5: "婚后分居",
    6: "二次起诉离婚",
    7: "按月给付抚养费",
    8: "准予离婚",
    9: "有夫妻共同债务",
    10: "婚前个人财产",
    11: "法定离婚",
    12: "不履行家庭义务",
    13: "存在非婚生子",
    14: "适当帮助",
    15: "不履行离婚协议",
    16: "损害赔偿",
    17: "感情不和分居满二年",
    18: "子女随非抚养权人生活",
    19: "婚后个人财产"
}

In [ ]:
# load_dataset()创建数据集
train_ds = load_dataset(read_custom_data, is_test=False, lazy=False) 
test_ds = load_dataset(read_custom_data, is_test=True, lazy=False)

# lazy=False，数据集返回为MapDataset类型
print("数据类型:", type(train_ds))

# labels为One-hot标签
print("训练集样例:", train_ds[0])
print("测试集样例:", test_ds[0])

### 2.3 加载中文ERNIE 3.0预训练模型和分词器

PaddleNLP中Auto模块（包括AutoModel, AutoTokenizer及各种下游任务类）提供了方便易用的接口，无需指定模型类别，即可调用不同网络结构的预训练模型。PaddleNLP的预训练模型可以很容易地通过from_pretrained()方法加载，[Transformer预训练模型汇总](https://paddlenlp.readthedocs.io/zh/latest/model_zoo/index.html#transformer)包含了40多个主流预训练模型，500多个模型权重。


AutoModelForSequenceClassification可用于多标签分类，通过预训练模型获取输入文本的表示，之后将文本表示进行分类。PaddleNLP已经实现了ERNIE 3.0预训练模型，可以通过一行代码实现ERNIE 3.0预训练模型和分词器的加载。

In [ ]:
# 加载中文ERNIE 3.0预训练模型和分词器
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "ernie-3.0-base-zh"
num_classes = 20
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_classes=num_classes)
tokenizer = AutoTokenizer.from_pretrained(model_name)

### 2.4 基于预训练模型的数据处理
`Dataset`中通常为原始数据，需要经过一定的数据处理并进行采样组batch。
* 通过`Dataset`的`map`函数，使用分词器将数据集从原始文本处理成模型的输入。
* 定义`paddle.io.BatchSampler`和`collate_fn`构建 `paddle.io.DataLoader`。

实际训练中，根据显存大小调整批大小`batch_size`和文本最大长度`max_seq_length`。

In [ ]:
import functools
import numpy as np

from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding

# 数据预处理函数，利用分词器将文本转化为整数序列
def preprocess_function(examples, tokenizer, max_seq_length):
    result = tokenizer(text=examples["text"], max_seq_len=max_seq_length)
    result["labels"] = examples["labels"]
    return result

trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128)
train_ds = train_ds.map(trans_func)
test_ds = test_ds.map(trans_func)

# collate_fn函数构造，将不同长度序列充到批中数据的最大长度，再将数据堆叠
collate_fn = DataCollatorWithPadding(tokenizer)

# 定义BatchSampler，选择批大小和是否随机乱序，进行DataLoader
train_batch_sampler = BatchSampler(train_ds, batch_size=64, shuffle=True)
test_batch_sampler = BatchSampler(test_ds, batch_size=64, shuffle=False)
train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
test_data_loader = DataLoader(dataset=test_ds, batch_sampler=test_batch_sampler, collate_fn=collate_fn)

### 2.5 数据训练和评估
定义训练所需的优化器、损失函数、评价指标等，就可以开始进行预模型微调任务。

In [ ]:
import time
import paddle.nn.functional as F

from demo.legalText.metric import MultiLabelReport

# Adam优化器、交叉熵损失函数、自定义MultiLabelReport评价指标
optimizer = paddle.optimizer.AdamW(learning_rate=1e-4, parameters=model.parameters())
criterion = paddle.nn.BCEWithLogitsLoss()
metric = MultiLabelReport()

In [ ]:
from demo.legalText.eval import evaluate
epochs = 10 # 训练轮次
ckpt_dir = "/home/work/PretrainedModel/legalText" #训练过程中保存模型参数的文件夹

global_step = 0 #迭代次数
tic_train = time.time()
best_f1_score = 0
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # 计算模型输出、损失函数值、分类概率值、准确率、f1分数
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.sigmoid(logits)
        metric.update(probs, labels)
        auc, f1_score, _, _ = metric.accumulate()

        # 每迭代10次，打印损失函数值、准确率、f1分数、计算速度
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, auc: %.5f, f1 score: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, auc, f1_score,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # 反向梯度回传，更新参数
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        # 每迭代40次，评估当前训练的模型、保存当前最佳模型参数和分词器的词表等
        if global_step % 40 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            eval_f1_score = evaluate(model, criterion, metric, test_data_loader, label_vocab, if_return_results=False)
            if eval_f1_score > best_f1_score:
                best_f1_score = eval_f1_score
                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)

模型训练过程中会输出如下日志:
```
global step 10, epoch: 1, batch: 10, loss: 0.34328, auc: 0.63276, f1 score: 0.22379, speed: 1.26 step/s
global step 20, epoch: 1, batch: 20, loss: 0.27681, auc: 0.68451, f1 score: 0.25070, speed: 0.90 step/s
global step 30, epoch: 1, batch: 30, loss: 0.21992, auc: 0.73419, f1 score: 0.29259, speed: 0.74 step/s
global step 40, epoch: 1, batch: 40, loss: 0.18223, auc: 0.78311, f1 score: 0.35254, speed: 0.62 step/s
eval loss: 0.17606, auc: 0.91074, f1 score: 0.72432, precison: 0.69068, recall: 0.76141
[2022-05-13 16:36:18,232] [    INFO] - tokenizer config file saved in ernie_ckpt_1/tokenizer_config.json
[2022-05-13 16:36:18,235] [    INFO] - Special tokens file saved in ernie_ckpt_1/special_tokens_map.json
...
```

### 2.6 多标签分类预测结果预测
加载微调好的模型参数进行情感分析预测，并保存预测结果

In [ ]:
from demo.legalText.eval import evaluate

# 可以加载训练好的模型参数结果查看模型训练结果
model.set_dict(paddle.load('/home/work/PretrainedModel/legalText/model_state.pdparams'))

# 模型在测试集中表现
print("ERNIE 3.0 在法律文本多标签分类test集表现", end= " ")
results = evaluate(model, criterion, metric, test_data_loader, label_vocab)

In [ ]:
test_ds = load_dataset(read_custom_data, is_test=True, is_one_hot=False, lazy=False)
res_dir = "/home/work/results/legalText"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
with open(os.path.join(res_dir, "multi_label.tsv"), 'w', encoding="utf8") as f:
    f.write("text\tprediction\n")
    for i, pred in enumerate(results):
        f.write(test_ds[i]['text']+"\t"+pred+"\n")

法律多标签文本预测结果示例:
![](https://ai-studio-static-online.cdn.bcebos.com/ddf3132abd7f4e3b9c29403a4624492b180d4bcc6eef4390ac962142ce276b13)

# 三.中文语义匹配实战

本项目将介绍如何基于PaddleNLP利用ERNIE 3.0预训练模型微调并进行中文语义匹配预测。本项目主要包括“什么是语义匹配任务”、“ERNIE 3.0模型”、“如何使用ERNIE 3.0中文预训练模型进行语义匹配”等三个部分。

## 1. 什么是语义匹配任务

文本语义匹配，通俗来讲就是判断两个文本的语义是否相同。文本语义匹配是自然语言处理中最基本的任务之一，语义匹配在搜索匹配、智能客服、新闻推荐等都有广泛的应用：在搜索匹配中，利用语义匹配技术，根据查询项检索与其相似内容；在智能客服中，通过将问题与候选答案或对话与回复相匹配，提高效率节约人工成本；在新闻推荐中，为用户推荐浏览过的新闻标题相似的新闻，提供新闻个性化推荐服务。

语义匹配任务常见两种训练范式:Point-wise和Pair-wise。其中单塔Point-wise匹配模型适合直接对文本对进行二分类的应用场景: 例如判断 2 个文本是否为语义相似；Pair-wise 匹配模型适合将文本对相似度作为特征之一输入到上层排序模块进行排序的应用场景。我们将示范如何使用ERNIE 3.0单塔Point-wise匹配模型在中文语义匹配数据集LCQMC进行微调。

![](https://ai-studio-static-online.cdn.bcebos.com/8330b7a380b347d7a087d5add94264deeb087f32aea84d9e8ce498dec9fa0869)
<font size=2><center>在百度搜索"什么是语义匹配"时会推荐相似搜索内容</center></font>

## 2. 如何使用ERNIE 3.0中文预训练模型进行语义匹配任务


### 2.1 环境准备

In [ ]:
# 安装pyzmq（若中文匹配中已经安装，这里可跳过）
!pip install pyzmq==18.1.1

### 2.2 加载中文语义匹配数据集LCQMC
LCQMC（Large-scale Chinese Question Matching Corpus）中文语义匹配数据集, 基于百度知道相似问题推荐构造的通问句中文语义匹配数据集，目的是为了解决在中文领域大规模问题匹配数据集的缺失。该数据集从百度知道不同领域的用户问题中抽取构建数据，数据集示例：
```
query    title    label
喜欢打篮球的男生喜欢什么样的女生	爱打篮球的男生喜欢什么样的女生	1
我手机丢了，我想换个手机	我想买个新手机，求推荐	1
大家觉得她好看吗	大家觉得跑男好看吗	0
晚上睡觉带着耳机听音乐有什么害处吗？	孕妇可以戴耳机听音乐吗?	0
```
其中1表示语义相似，0表示语义不相似，PaddleNLP已经内置该数据集，一键即可加载。

In [ ]:
# 加载中文语义匹配数据集lcqmc
from paddlenlp.datasets import load_dataset

train_ds, dev_ds, test_ds = load_dataset("lcqmc", splits=["train", "dev", "test"])

# 数据集返回为MapDataset类型
print("数据类型:", type(train_ds))
# label代表标签，测试集中不包含标签信息
print("训练集样例:", train_ds[0])
print("验证集样例:", dev_ds[0])
print("测试集样例:", test_ds[0])

### 2.3 加载中文ERNIE 3.0预训练模型和分词器

PaddleNLP中Auto模块（包括AutoModel, AutoTokenizer及各种下游任务类）提供了方便易用的接口，无需指定模型类别，即可调用不同网络结构的预训练模型。PaddleNLP的预训练模型可以很容易地通过from_pretrained()方法加载，[Transformer预训练模型汇总](https://paddlenlp.readthedocs.io/zh/latest/model_zoo/index.html#transformer)包含了40多个主流预训练模型，500多个模型权重。


AutoModelForSequenceClassification可用于Point-wise方式的二分类语义匹配任务，通过预训练模型获取输入文本对（query-title）的表示，之后将文本表示进行分类。PaddleNLP已经实现了ERNIE 3.0预训练模型，可以通过一行代码实现ERNIE 3.0预训练模型和分词器的加载。

In [ ]:
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "ernie-3.0-base-zh"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_classes=len(train_ds.label_list))
tokenizer = AutoTokenizer.from_pretrained(model_name)

### 2.4 基于预训练模型的数据处理
`Dataset`中通常为原始数据，需要经过一定的数据处理并进行采样组batch。
* 通过`Dataset`的`map`函数，使用分词器将数据集中query文本和title文本拼接，从原始文本处理成模型的输入。
* 定义`paddle.io.BatchSampler`和`collate_fn`构建 `paddle.io.DataLoader`。

实际训练中，根据显存大小调整批大小`batch_size`和文本最大长度`max_seq_length`。

In [ ]:
import functools
import numpy as np

from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding

# 数据预处理函数，利用分词器将文本转化为整数序列
def preprocess_function(examples, tokenizer, max_seq_length, is_test=False):
    
    result = tokenizer(text=examples["query"], text_pair=examples["title"], max_seq_len=max_seq_length)
    if not is_test:
        result["labels"] = examples["label"]
    return result

trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128)
train_ds = train_ds.map(trans_func)
dev_ds = dev_ds.map(trans_func)

# collate_fn函数构造，将不同长度序列充到批中数据的最大长度，再将数据堆叠
collate_fn = DataCollatorWithPadding(tokenizer)

# 定义BatchSampler，选择批大小和是否随机乱序，进行DataLoader
train_batch_sampler = BatchSampler(train_ds, batch_size=64, shuffle=True)
dev_batch_sampler = BatchSampler(dev_ds, batch_size=128, shuffle=False)
train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
dev_data_loader = DataLoader(dataset=dev_ds, batch_sampler=dev_batch_sampler, collate_fn=collate_fn)

### 2.5 数据训练和评估
定义训练所需的优化器、损失函数、评价指标等，就可以开始进行预模型微调任务。

In [ ]:
# Adam优化器、交叉熵损失函数、accuracy评价指标
optimizer = paddle.optimizer.AdamW(learning_rate=5e-5, parameters=model.parameters())
criterion = paddle.nn.loss.CrossEntropyLoss()
metric = paddle.metric.Accuracy()

In [ ]:
# 开始训练
import time
import paddle.nn.functional as F

from demo.semanticMatchCn.eval import evaluate

epochs = 1 # 训练轮次
ckpt_dir = "/home/work/PretrainedModel/semanticMatchCn" #训练过程中保存模型参数的文件夹
best_acc = 0
best_step = 0
global_step = 0 #迭代次数
tic_train = time.time()
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # 计算模型输出、损失函数值、分类概率值、准确率
        logits = model(input_ids, token_type_ids)
        loss = criterion(logits, labels)
        probs = F.softmax(logits, axis=1)
        correct = metric.compute(probs, labels)
        metric.update(correct)
        acc = metric.accumulate()

        # 每迭代10次，打印损失函数值、准确率、计算速度
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, acc,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # 反向梯度回传，更新参数
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()

        # 每迭代100次，评估当前训练的模型、保存当前最佳模型参数和分词器的词表等
        if global_step % 100 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            print("global step", global_step, end=' ')
            acc_eval = evaluate(model, criterion, metric, dev_data_loader)
            if acc_eval > best_acc:
                best_acc = acc_eval
                best_step = global_step

                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)

模型训练过程中会输出如下日志:
```
global step 10, epoch: 1, batch: 10, loss: 0.55404, accu: 0.64844, speed: 5.08 step/s
global step 20, epoch: 1, batch: 20, loss: 0.29190, accu: 0.73125, speed: 5.36 step/s
global step 30, epoch: 1, batch: 30, loss: 0.37968, accu: 0.76719, speed: 5.51 step/s
global step 40, epoch: 1, batch: 40, loss: 0.34762, accu: 0.79453, speed: 5.26 step/s
global step 50, epoch: 1, batch: 50, loss: 0.29682, accu: 0.80969, speed: 5.03 step/s
global step 60, epoch: 1, batch: 60, loss: 0.24128, accu: 0.82344, speed: 5.07 step/s
global step 70, epoch: 1, batch: 70, loss: 0.24660, accu: 0.83058, speed: 5.35 step/s
global step 80, epoch: 1, batch: 80, loss: 0.32774, accu: 0.83828, speed: 5.46 step/s
global step 90, epoch: 1, batch: 90, loss: 0.40908, accu: 0.84306, speed: 5.55 step/s
global step 100, epoch: 1, batch: 100, loss: 0.38926, accu: 0.84750, speed: 5.18 step/s
global step 100 eval dev loss: 0.35993, accu: 0.84276
[2022-05-13 16:54:52,751] [    INFO] - tokenizer config file saved in ernie_ckpt_1/tokenizer_config.json
[2022-05-13 16:54:52,754] [    INFO] - Special tokens file saved in ernie_ckpt_1/special_tokens_map.json
...
```
在单卡运行环境中，一个epoch大约需要22分钟

In [ ]:
from demo.semanticMatchCn.eval import evaluate

# 加载训练好的模型参数结果查看模型训练结果
params_path = '/home/work/PretrainedModel/semanticMatchCn/model_state.pdparams'
state_dict = paddle.load(params_path)
model.set_dict(state_dict)
print('ERNIE 3.0 在lcqmc的dev集表现', end=' ')
eval_acc = evaluate(model, criterion, metric, dev_data_loader)

### 2.6 语义匹配结果预测与保存
加载微调好的模型参数进行语义匹配预测，并保存预测结果

In [ ]:
# 测试集数据预处理，利用分词器将文本转化为整数序列
trans_func_test = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128, is_test=True)
test_ds_trans = test_ds.map(trans_func_test)

# 进行采样组batch
collate_fn_test = DataCollatorWithPadding(tokenizer)
test_batch_sampler = BatchSampler(test_ds_trans, batch_size=32, shuffle=False)
test_data_loader = DataLoader(dataset=test_ds_trans, batch_sampler=test_batch_sampler, collate_fn=collate_fn_test)

In [ ]:
# 模型预测分类结果
import paddle.nn.functional as F

label_map = {0: '不相似', 1: '相似'}
results = []
model.eval()
for batch in test_data_loader:
    input_ids, token_type_ids = batch['input_ids'], batch['token_type_ids']
    logits = model(batch['input_ids'], batch['token_type_ids'])
    probs = F.softmax(logits, axis=-1)
    idx = paddle.argmax(probs, axis=1).numpy()
    idx = idx.tolist()
    preds = [label_map[i] for i in idx]
    results.extend(preds)

In [ ]:
# 存储LCQMC预测结果
test_ds = load_dataset("lcqmc", splits=["test"])    
res_dir = "/home/work/results/semanticMatchCn"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
with open(os.path.join(res_dir, "lcqmc.tsv"), 'w', encoding="utf8") as f:
    f.write("label\tquery\ttitle\n")
    for i, pred in enumerate(results):
        f.write(pred+"\t"+test_ds[i]['query']+"\t"+test_ds[i]['title']+"\n")

LCQMC预测存储结果:

![](https://ai-studio-static-online.cdn.bcebos.com/210b21c8d9154320839bbd47541be9c80dd1dfd381e24d00b52c4da5799114cf)


# 四.MSRA序列标注实战

本项目将介绍如何基于PaddleNLP利用ERNIE 3.0预训练模型微调并进行中文序列标注预测。本项目主要包括“什么是序列标注”、“ERNIE 3.0模型”、“如何使用ERNIE 3.0中文预训练模型进行MSRA序列标注”等三个部分。

## 1. 什么是序列标注

序列标注(Sequence Tagging)是经典的自然语言处理问题，可以用于解决一系列字符分类问题，例如分词、词性标注(POS tagging)、命名实体识别(Named Entity Recognition，NER)、关键词抽取、语义角色标注(Semantic Role Labeling)、槽位抽取(Slot Filling)。在现实场景中，序列标注技术可以帮助完成简历、快递单、病例医疗实体信息抽取等。

在序列标注任务中，一般会定义一个标签集合来表示所有预测结果。对于输入的序列，任务目标是对序列中所有字符进行标记。在深度学习中，通常将序列标注问题视为分类问题，对输入序列的每一个token进行一次多分类任务进行训练预测。

![](https://ai-studio-static-online.cdn.bcebos.com/7f931b66e5fc4e4eaf8d1dc60a950413c16b7188380a480c9c5dba2ec768a4fb)


<font size=2><center>对输入序列进行标记</center></font>

## 2. 如何使用ERNIE 3.0中文预训练模型进行MSRA序列标注


### 2.1 环境准备

In [ ]:
# 若“法律文本多标签”或“中文语义匹配”项目已安装，则可跳过此步骤
!pip install pyzmq==18.1.1

### 2.2 加载MSRA-NER数据集

MSRA-NER 数据集由微软亚研院发布，其目标是识别文本中具有特定意义的实体，主要包括人名、地名、机构名等。PaddleNLP已经内置该数据集，一键即可加载。PaddleNLP集成的数据集MSRA-NER数据集对文件格式做了调整：每一行文本、标签以特殊字符"\t"进行分隔，每个字之间以特殊字符"\002"分隔。示例如下：
```
不\002久\002前\002，\002中\002国\002共\002产\002党\002召\002开\002了\002举\002世\002瞩\002目\002的\002第\002十\002五\002次\002全\002国\002代\002表\002大\002会\002。O\002O\002O\002O\002B-ORG\002I-ORG\002I-ORG\002I-ORG\002I-ORG\002O\002O\002O\002O\002O\002O\002O\002O\002B-ORG\002I-ORG\002I-ORG\002I-ORG\002I-ORG\002I-ORG\002I-ORG\002I-ORG\002I-ORG\002I-ORG\002O

这\002次\002代\002表\002大\002会\002是\002在\002中\002国\002改\002革\002开\002放\002和\002社\002会\002主\002义\002现\002代\002化\002建\002设\002发\002展\002的\002关\002键\002时\002刻\002召\002开\002的\002历\002史\002性\002会\002议\002。O\002O\002O\002O\002O\002O\002O\002O\002B-LOC\002I-LOC\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O\002O
```

加载MSRA_NER数据集为BIO标注集：
* B-PER、I-PER代表人名首字、人名非首字。
* B-LOC、I-LOC代表地名首字、地名非首字。
* B-ORG、I-ORG代表组织机构名首字、组织机构名非首字。
* O代表该字不属于命名实体的一部分。

In [ ]:
# 加载MSRA_NER数据集
from paddlenlp.datasets import load_dataset

train_ds, test_ds = load_dataset('msra_ner', splits=('train', 'test'), lazy=False)
label_vocab = {label:label_id for label_id, label in enumerate(train_ds.label_list)}

# 数据集返回类型为MapDataset
print("数据类型:", type(train_ds))
print("数据标签:", label_vocab)

# 每条数据包含一句文本和这个文本中每个汉字以及数字对应的label标签
print("训练集样例:", train_ds[0])
print("测试集样例:", test_ds[0])

### 2.3 加载中文ERNIE 3.0预训练模型和分词器

PaddleNLP中[Auto模块](https://paddlenlp.readthedocs.io/zh/latest/source/paddlenlp.transformers.auto.modeling.html)（包括AutoModel, AutoTokenizer及各种下游任务类）提供了方便易用的接口，无需指定模型类别，即可调用不同网络结构的预训练模型。PaddleNLP的预训练模型可以很容易地通过from_pretrained()方法加载，[Transformer预训练模型汇总](https://paddlenlp.readthedocs.io/zh/latest/model_zoo/index.html#transformer)包含了40多个主流预训练模型，500多个模型权重。


`AutoForTokenClassification`可用于序列标注，通过预训练模型获取输入文本每个token的表示，之后将token表示进行分类。PaddleNLP已经实现了ERNIE 3.0预训练模型，可以通过一行代码实现ERNIE 3.0预训练模型和分词器的加载。

In [ ]:
from paddlenlp.transformers import AutoModelForTokenClassification
from paddlenlp.transformers import AutoTokenizer

model_name = "ernie-3.0-base-zh"
model = AutoModelForTokenClassification.from_pretrained(model_name, num_classes=len(train_ds.label_list))
tokenizer = AutoTokenizer.from_pretrained(model_name)

### 2.4 基于预训练模型的数据处理
`Dataset`中通常为原始数据，需要经过一定的数据处理转成可输入模型的数据并进行采样组batch。
* 通过`Dataset`的`map`函数，使用分词器将数据集从原始文本处理成模型的输入。
* 定义`paddle.io.BatchSampler`和`collate_fn`构建 `paddle.io.DataLoader`。

实际训练中，根据显存大小调整批大小`batch_size`和文本最大长度`max_seq_length`。

In [ ]:
import functools
import numpy as np

from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorForTokenClassification

# 数据预处理函数，利用分词器将文本转化为整数序列
def preprocess_function(example, tokenizer, label_vocab, max_seq_length=128):

    labels = example['labels']
    tokens = example['tokens']
    no_entity_id = label_vocab['O']

    tokenized_input = tokenizer(tokens, return_length=True, is_split_into_words=True, max_seq_len=max_seq_length)

    # 保证label与input_ids长度一致
    # -2 for [CLS] and [SEP]
    if len(tokenized_input['input_ids']) - 2 < len(labels):
        labels = labels[:len(tokenized_input['input_ids']) - 2]
    tokenized_input['labels'] = [no_entity_id] + labels + [no_entity_id]
    tokenized_input['labels'] += [no_entity_id] * (len(tokenized_input['input_ids']) - len(tokenized_input['labels'])) 
    
    return tokenized_input


trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, label_vocab=label_vocab, max_seq_length=128)
train_ds = train_ds.map(trans_func)
test_ds = test_ds.map(trans_func)

# collate_fn函数构造，将不同长度序列充到批中数据的最大长度，再将数据堆叠
collate_fn = DataCollatorForTokenClassification(tokenizer=tokenizer, label_pad_token_id=-1)

# 定义BatchSampler，选择批大小和是否随机乱序，进行DataLoader
train_batch_sampler = BatchSampler(train_ds, batch_size=32, shuffle=True)
test_batch_sampler = BatchSampler(test_ds, batch_size=32, shuffle=False)
train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
test_data_loader = DataLoader(dataset=test_ds, batch_sampler=test_batch_sampler, collate_fn=collate_fn)

### 2.5 数据训练和评估
定义训练所需的优化器、损失函数、评价指标等，就可以开始进行预模型微调任务。

In [ ]:
from paddlenlp.metrics import ChunkEvaluator

# Adam优化器、交叉熵损失函数、ChunkEvaluator评价指标
optimizer = paddle.optimizer.AdamW(learning_rate=2e-5, parameters=model.parameters())
criterion = paddle.nn.loss.CrossEntropyLoss(ignore_index=-1)
metric = ChunkEvaluator(label_list=train_ds.label_list)

In [ ]:
# 开始训练
import time
import paddle.nn.functional as F

from demo.sequenceTagging.utils import evaluate

epochs = 10 # 训练轮次
ckpt_dir = "/home/work/PretrainedModel/sequenceTagging" #训练过程中保存模型参数的文件夹
best_f1_score = 0
best_step = 0
global_step = 0 #迭代次数
tic_train = time.time()
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # 计算模型输出、损失函数值
        logits = model(input_ids, token_type_ids)
        loss =  paddle.mean(criterion(logits, labels))

        # 每迭代10次，打印损失函数值、计算速度
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, 10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # 反向梯度回传
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()

        # 每迭代200次，评估当前训练的模型、保存当前最佳模型参数和分词器的词表等
        if global_step % 200 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            print('global_step', global_step, end=' ')
            f1_score_eval = evaluate(model, metric, test_data_loader)
            if f1_score_eval > best_f1_score:
                best_f1_score = f1_score_eval
                best_step = global_step

                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)

模型每训练迭代100次会评估一次，训练过程中会输出如下日志:
```
global step 10, epoch: 1, batch: 10, loss: 0.69060, speed: 5.02 step/s
global step 20, epoch: 1, batch: 20, loss: 0.57494, speed: 5.07 step/s
global step 30, epoch: 1, batch: 30, loss: 0.55177, speed: 5.18 step/s
global step 40, epoch: 1, batch: 40, loss: 0.44063, speed: 4.91 step/s
global step 50, epoch: 1, batch: 50, loss: 0.19791, speed: 5.00 step/s
global step 60, epoch: 1, batch: 60, loss: 0.28906, speed: 5.02 step/s
global step 70, epoch: 1, batch: 70, loss: 0.17547, speed: 5.18 step/s
global step 80, epoch: 1, batch: 80, loss: 0.17136, speed: 4.98 step/s
global step 90, epoch: 1, batch: 90, loss: 0.20389, speed: 4.90 step/s
global step 100, epoch: 1, batch: 100, loss: 0.14262, speed: 4.89 step/s
global step 110, epoch: 1, batch: 110, loss: 0.09750, speed: 5.26 step/s
global step 120, epoch: 1, batch: 120, loss: 0.16538, speed: 5.09 step/s
global step 130, epoch: 1, batch: 130, loss: 0.10952, speed: 4.93 step/s
global step 140, epoch: 1, batch: 140, loss: 0.16549, speed: 5.21 step/s
global step 150, epoch: 1, batch: 150, loss: 0.14116, speed: 5.15 step/s
global step 160, epoch: 1, batch: 160, loss: 0.10677, speed: 4.90 step/s
global step 170, epoch: 1, batch: 170, loss: 0.05065, speed: 5.30 step/s
global step 180, epoch: 1, batch: 180, loss: 0.14484, speed: 4.99 step/s
global step 190, epoch: 1, batch: 190, loss: 0.07996, speed: 4.92 step/s
global step 200, epoch: 1, batch: 200, loss: 0.10816, speed: 5.11 step/s
global_step 200 eval precision: 0.759136 - recall: 0.855165 - f1: 0.804294
[2022-05-13 16:05:59,091] [    INFO] - tokenizer config file saved in ernie_ckpt_1/tokenizer_config.json
[2022-05-13 16:05:59,094] [    INFO] - Special tokens file saved in ernie_ckpt_1/special_tokens_map.json
...
```
10个epoch预计训练时间60分钟。

### 2.6 序列标注结果预测与保存
加载微调好的模型参数进行情感分析预测，并保存预测结果

In [ ]:
# 测试集结果评估
from demo.sequenceTagging.utils import parse_decodes

# 加载训练好的模型参数结果查看模型训练结果
# 加载最佳模型参数
model.set_dict(paddle.load('/home/work/PretrainedModel/sequenceTagging/model_state.pdparams'))
model.eval()
metric.reset()

pred_list = []
len_list = []

for step, batch in enumerate(test_data_loader, start=1):
    input_ids, token_type_ids, labels, lens = batch['input_ids'], batch['token_type_ids'], batch['labels'], batch['seq_len']

    logits = model(input_ids, token_type_ids)
    preds = paddle.argmax(logits, axis=-1)

    n_infer, n_label, n_correct = metric.compute(lens, preds, labels)
    metric.update(n_infer.numpy(), n_label.numpy(), n_correct.numpy())

    pred_list.append(preds.numpy())
    len_list.append(lens.numpy())

precision, recall, f1_score = metric.accumulate()
print("ERNIE 3.0 在msra_ner的test集表现 -precision: %f - recall: %f - f1: %f" %(precision, recall, f1_score))

我们根据模型预测结果对文本进行后处理，对文本序列进行标注，具体的标签含义如下：
* ‘O’: no special entity(其他不属于任何实体的字符,包括标点等)
* ‘PER’: person(人名)
* ‘ORG’: organization(组织机构)
* ‘LOC’: location（地点）

In [ ]:
# 根据预测结果对文本进行后处理
test_ds = load_dataset('msra_ner', splits=('test')) 
preds = parse_decodes(test_ds, pred_list, len_list, label_vocab)

# 查看预测结果
print("查看结果")
print("\n".join(preds[:2]))

# 保存预测结果
res_dir = "/home/work/results/sequenceTagging"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
with open(os.path.join(res_dir, "results.txt"), "w", encoding="utf-8") as f:
    f.write("\n".join(preds))

## 温馨提示
每个项目训练完成生成模型文件后，若想保存当前模型版本，需要将生成的模型文件移动到/home/work/PretrainedModel目录下。